## Exercise 1 - Depth map to point cloud
* Load the RGB image and depth map, and use them to generate a colored point cloud (see this Open3D tutorial for pointers.) You can assume a pinhole camera model with the following intrinsic matrix:
[[572.4114, 0.0, 325.2611], [ 0.0, 573.57043, 242.04899], [ 0.0, 0.0, 1.0]]
* Visualize the resulting point cloud and save it as a file.

* Write a function that returns the 3D coordinate of a given pixel (u,v) using the intrinsic parameters.

* Define a bounding box around an object of your choice in the image, convert the corner locations to 3D coordinates and use them to crop the point cloud. You can use the PointCloud.crop() function in Open3D.

In [1]:
# Importing libraries
import numpy as np
import cv2 as cv
import os
import open3d as o3d

# Defining data paths
data_path = "../Data/"
output_path = "Output/"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
print("Read Redwood dataset")
color_raw = o3d.io.read_image(data_path+"rgb.png")
depth_raw = o3d.io.read_image(data_path+"depth.png")
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw)
print(rgbd_image)